<hr style="border:3px solid black"></hr>
Contact : dhilip@iitrpr.ac.in <br>

    - Apoorv Kushwaha & Dr. T.J. Dhilip Kumar
        Quantum Dynamics Lab (410), IIT Ropar.
***
# 4D SF expansion code for collision of 2 rigid rotors:

### Install instructions:
#### Steps to install anaconda 
1. Install anaconda from [here](https://www.anaconda.com/) 
* It is recommended to __create a new environment in anaconda to prevent any conflict__ between existing libraries and pyshtools. <br> Conflicts can take hours to resolve or may fail completely after hours of wait.
#### Quick Install
1. Use PES2MP_quick.sh to install PES2MP_quick environment in anaconda. (Instructions are available on GitHub page!)
### Install Instructions for standalone code (Pyshtools Requires: Python >=3.6)
2. The library pyshtools can be installed locally using conda from this [website](https://pypi.org/project/pyshtools/). 
2. Open command prompt and enter *(One time only)*:
    1. conda create -n pysh <span style="color:green">*--creates a new clean environment pysh--*</span>
    2. conda activate pysh  <span style="color:green">*-- activates the environment pysh--*</span>
    3. conda install -c conda-forge pyshtools <span style="color:green">*--installs the pyshtools library inside pysh environment--*</span>
    3. conda install -c conda-forge tqdm <span style="color:green">--optional, read below--</span>
3. After installation open command prompt and enter. 
    4. conda activate pysh
    5. jupyter-notebook <span style="color:green">*--Opens jupyter notebook. Now locate the required notebook in jupyter and run--* </span>
* The needed __time for computing spherical harmonics term is large (several hours)__. <br> Therefore, do not run the file in google colab but preferebly run locally by installing anaconda.
4. [tqdm](https://pypi.org/project/tqdm/#installation) shows progress for any loop __(optional but useful)__


<hr style="border:2px solid black"></hr>

### *Prepare input data ---> Import libraries ---> Create required folders*

    - Example Input  : Make Required changes in the cell as needed. [green]
    - Example Output : Describes the output of the cell. [indigo]

<hr style="border:2px solid black"></hr>

#### Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from math import sin, cos, radians
from sympy.physics.wigner import clebsch_gordan
import os
import pyshtools as pysh
from tqdm.notebook import tqdm
import time

In [2]:
# creating required folders
input_dir = os.getcwd()+'/'

out_plots = input_dir + 'plots/4D_MP/'
if not os.path.exists(out_plots):
    os.makedirs(out_plots)
out_data = input_dir + 'data/4D_MP/'
if not os.path.exists(out_data):
    os.makedirs(out_data)

<hr style="border:2px solid black"></hr>

### Importing input 4D PES file (E_4D_ml.csv) with descriptors as R $,\phi, \theta_2, \theta_1$ and E.
Input file is imported and sorted (transformed) as needed. 

   <u>Example Input</u>

    Input name: 2_inp_4D_PES.dat      # file name with R, phi, th2, th1 and E
    Separation: \s+                   # \s+ : multiple spaces in separation. Other options are comma (,) and tab (\t), etc. 
    names: 'R','phi','th2','th1','E'  # input file should not contain any header. 

    *E_inf = NOT NEEDED (input is already in cm-1)
***
<span style="color:indigo"><u> Example Output</u></span>
   
       Prints contents of df_inp (DataFrame containing transformed 4D input PES) 
***
Use commented lines to convert energy to cm-1 (Required! If input energies are in Hartree). The input PES data is sorted by R followed by rest of the angular terms. No data point must be missing. If so, use CODE 0 and 1 to obtain those points.
<hr style="border:2px solid black"></hr>

In [3]:
df_inp = pd.read_csv('2_inp_4D_PES.dat',header=None,sep='\s+',names=['R','phi','th2','th1','E']) 
df_inp.sort_values(by = [ 'R','phi','th2','th1'], inplace=True, ascending = True)
df_inp.reset_index(inplace=True, drop = True)  # sorting by R, phi, th2 and th1 and reindexing

# Use below code to convert Hartree in cm^{-1}. 
#E_inf = 186.5889233299                         # Set Asymptotic (R) energy
#df_inp['E'] = (df_inp['E'] + E_inf)*219474.63  # converting to cm-1
print(df_inp[:10])                              # printing df_inp to see input file. 

     R  phi  th2  th1              E
0  2.5    0    0    0  424269.734133
1  2.5    0    0   30   98040.468659
2  2.5    0    0   60    6253.008116
3  2.5    0    0   90    1281.146077
4  2.5    0    0  120    6323.170408
5  2.5    0    0  150   97282.681157
6  2.5    0    0  180  424251.254699
7  2.5    0   30    0  390680.819034
8  2.5    0   30   30   93583.895927
9  2.5    0   30   60    6155.878359


<hr style="border:2px solid black"></hr>

### Extracting matrix contining angular coordinates
- It is important to note that input energies must be sorted by R followed by $\phi, \theta_2$ and $\theta_1$.
- If you have poor distribution of angular coordinates, use CODE 0 and 1 to get new coordinates at regular interval.

***
<span style="color:indigo"><u> Example Output</u></span>
   
       Prints matrix containing angular coordinates.
<hr style="border:2px solid black"></hr>


In [4]:
angmat = df_inp[['phi','th2','th1']].drop_duplicates().to_numpy() # extracting unique angular coordinates
print("Number of angular terms: ", len(angmat))
print("Angular terms will be saved in /Data/4D_MP/Ang_Mat.dat")
np.savetxt(out_data +"Ang_Mat.dat",angmat,fmt='%i\t%i\t%i') 
print(angmat)

Number of angular terms:  112
Angular terms will be saved in /Data/4D_MP/Ang_Mat.dat
[[  0   0   0]
 [  0   0  30]
 [  0   0  60]
 [  0   0  90]
 [  0   0 120]
 [  0   0 150]
 [  0   0 180]
 [  0  30   0]
 [  0  30  30]
 [  0  30  60]
 [  0  30  90]
 [  0  30 120]
 [  0  30 150]
 [  0  30 180]
 [  0  60   0]
 [  0  60  30]
 [  0  60  60]
 [  0  60  90]
 [  0  60 120]
 [  0  60 150]
 [  0  60 180]
 [  0  90   0]
 [  0  90  30]
 [  0  90  60]
 [  0  90  90]
 [  0  90 120]
 [  0  90 150]
 [  0  90 180]
 [ 30   0   0]
 [ 30   0  30]
 [ 30   0  60]
 [ 30   0  90]
 [ 30   0 120]
 [ 30   0 150]
 [ 30   0 180]
 [ 30  30   0]
 [ 30  30  30]
 [ 30  30  60]
 [ 30  30  90]
 [ 30  30 120]
 [ 30  30 150]
 [ 30  30 180]
 [ 30  60   0]
 [ 30  60  30]
 [ 30  60  60]
 [ 30  60  90]
 [ 30  60 120]
 [ 30  60 150]
 [ 30  60 180]
 [ 30  90   0]
 [ 30  90  30]
 [ 30  90  60]
 [ 30  90  90]
 [ 30  90 120]
 [ 30  90 150]
 [ 30  90 180]
 [ 60   0   0]
 [ 60   0  30]
 [ 60   0  60]
 [ 60   0  90]
 [ 60   0 120]


<hr style="border:2px solid black"></hr>

### Creating the matrix contining order of radial terms $\lambda_1, \lambda_2, \& \ \lambda $
* Number of radial terms (for each species) must be <= Number of angular terms (for each species)

<span style="color:green"><u> Example Input</u></span>
   
        L1max  = 6                           # max order for first radial term (NCCN)
        L2max  = 4                           # max order for second radial term (H_2)
        Symm_1 = 2                           # 1 for non symm, 2 for symmetric (NCCN is symmetric)
        Symm_2 = 2                           # 1 for non symm, 2 for symmetric (H2 is symmetric)
        Symm   = min(Symm_1,Symm_2)          # If both Symm_1 and Symm_2 are 2 (symmetric), use 2 else 1.

Symm = min(Symm_1,Symm_2) is also an alternative but I guess users must be aware about step size of Lambda.
***
<span style="color:indigo"><u> Example Output</u></span>
    
    Prints matrix (Lmat) containing lambda terms and saves in /Data/4D_MP/Lambda_ref.dat
***
An extra column (pointer) with series of whole number (0-27) is added to identify terms in Lmat matrix.  

  - The output is saved in "data/4D_MP" folder. <br>
  - First column is the pointer for reference (an integer),<br>
  - Second column is $\lambda_1$<br>
  - Third column is $\lambda_2$<br>
  - Fourth column is $\lambda$<br>

<hr style="border:2px solid black"></hr>

In [5]:
Lmat = np.zeros((1,3))
L1max = 6                           # max order for radial term (NCCN)
L2max = 4                           # max order for radial term (H_2)
Symm_1 = 2                          # 1 for non symm, 2 for symm
Symm_2 = 2                          # 1 for non symm, 2 for symm
Symm = min(Symm_1,Symm_2)           # 1 if none of the above is symm, else 2

for i in range(0,L1max+1,Symm_1):        # loop for Lambda_1 
    for j in range(0,L2max+1,Symm_1):    # loop for Lambda_2 
        for k in range(abs(i-j),abs(i+j)+1,Symm):    # loop for Lambda 
            Lc = np.matrix([i,j,k])
            Lmat = np.append(Lmat,Lc,axis=0)
Lmat = np.delete(Lmat,0,0)
Lmat = Lmat.astype(int)
print("Number of radial terms: ", len(Lmat))

# appending Lambda matrix with integers that will point to specific radial coefficient.
num   = np.arange(len(Lmat))
num2  = np.reshape(num, (-1, 1))
Lmat1 = np.append(num2,Lmat,axis=1)
Lmat1   # Vlam along with their numbering
print("Lambda terms will be saved in /Data/Lambda_ref.dat")
print(Lmat1)
# saving radial coefficients for future reference (The final data does not contain V_lambda terms but the pointer in the first column)
np.savetxt(out_data +"Lambda_ref.dat",Lmat1,fmt='%i\t%i\t%i\t%i') 

Number of radial terms:  28
Lambda terms will be saved in /Data/Lambda_ref.dat
[[ 0  0  0  0]
 [ 1  0  2  2]
 [ 2  0  4  4]
 [ 3  2  0  2]
 [ 4  2  2  0]
 [ 5  2  2  2]
 [ 6  2  2  4]
 [ 7  2  4  2]
 [ 8  2  4  4]
 [ 9  2  4  6]
 [10  4  0  4]
 [11  4  2  2]
 [12  4  2  4]
 [13  4  2  6]
 [14  4  4  0]
 [15  4  4  2]
 [16  4  4  4]
 [17  4  4  6]
 [18  4  4  8]
 [19  6  0  6]
 [20  6  2  4]
 [21  6  2  6]
 [22  6  2  8]
 [23  6  4  2]
 [24  6  4  4]
 [25  6  4  6]
 [26  6  4  8]
 [27  6  4 10]]


<hr style="border:2px solid black"></hr>

### Initializing required variables
***
    R_arr = df_inp['R'].unique()   # extracting unique values of R (sorted)
    
    If the header for 'R' is changed, the same must be changed here!

<hr style="border:2px solid black"></hr>

In [7]:
lm = len(Lmat)               # number of lambda (radial) terms
ngm = len(angmat)            # number of angular terms
px = np.zeros((ngm,lm))      # initializing matrix to store BiSp coefficients
R_arr = df_inp['R'].unique()   # extracting unique values of R (sorted)
Rpt = len(R_arr)             # number of radial (R) terms
f = np.zeros(ngm)            # 1D matrix to store energies
V_nf = np.zeros((Rpt,lm))    # 1D matrix to store least sq. fit terms

<hr style="border:2px solid black"></hr>

## Function to calculate bispherical harmonics coefficients (using Pyshtools)

* **BiSp:** bispherical harmonics coefficient = $Y_lm(\theta_1,0) * Y_lm(\theta_2,\phi) *$ CG coefficient [Expanded form is given below]

$\left (\frac{2 \lambda+1}{4 \pi}\right)^{1 / 2} \times 
[ \left\langle\lambda_{1}, 0, \lambda_{2},0 \mid \lambda, 0\right\rangle 
P_{\lambda_1}^{0}\left(\theta_1, 0\right) P_{\lambda_2}^{0} 
\sum_{m=1}^{l_m} (-1)^{m} \times 2 \times \left\langle\lambda_{1}, m, \lambda_{2},-m \mid \lambda, 0\right\rangle 
\quad P_{\lambda_1}^{m}\left(\theta_1, 0\right) P_{\lambda_2}^{-m}\left(\theta_2, \phi\right)\times cos (m\phi)]$
<hr style="border:2px solid black"></hr>


In [ ]:
def Bispher_SF(L1,L2,L,ph,th2,th1):
    Total=0
    M_max= min(L1,L2)
    theta1 = radians(th1)
    theta2 = radians(th2)
    phi = radians(ph)
    
    U00 = np.sqrt((2*L+1)/(4*np.pi))
    T01 = clebsch_gordan(L1, L2, L, 0, 0, 0).evalf()
    P0L1 = pysh.legendre.legendre_lm (L1, 0, cos(theta1), 'unnorm', -1, 0)
    P0L2 = pysh.legendre.legendre_lm (L2, 0, cos(theta2), 'unnorm', -1, 0)
    T1 = T01*P0L1*P0L2
    Total += T1
    M=1
    while (M<=M_max):
        T02 = clebsch_gordan(L1, L2, L, M, -M, 0).evalf()
        PmL1 = pysh.legendre.legendre_lm (L1, M, cos(theta1), 'unnorm', -1, 0)
        PmL2 = pysh.legendre.legendre_lm (L2, M, cos(theta2), 'unnorm', -1, 0)
        T2 = T02*PmL1*PmL2
        Total += 2.0*pow((-1),M)*T2*(cos(M*phi))
        M+=1
    Total=Total*U00
    return Total

<hr style="border:2px solid black"></hr>

### Running loop over full range of angular terms to calculate bispherical harmonics coefficients

* px stores the bispherical harmonics coefficients and C_inv stores the pseudo_inverse of the same.
* The V_lambda coefficients (and pseudo-inverse) are available for all combination of thetas and phi (sorted)
* _One can uses least sq. fit for solving the matrix (which is more intuitive but equivalent to pseudoinverse), 
the same is given below_<br>
 <span style="color:red">V_n1 = np.linalg.lstsq(px, f, rcond=None)[0]</span> 
 <span style="color:green">_--pseudoinverse = least sq. fit (Moore–Penrose inverse)--_</span>
***
        TWO CHOICES for bispherical harmonics coefficients:
        1) Run the next tab to generate new coefficients (Warning: It can take enormous time to compute new coefficients).
        2) Skip the next tab and run the subsequent (Next+1) tab to load previously generated coefficients from a numpy file. 
***
* tqdm is implemented to show remaining time in computing new coefficients.
***
<span style="color:indigo"><u> Example output for next cell</u></span>

    Bispherical coefficients (px) are calculated and automatically saved to /data/4D_MP/4D_BiSp_coeff.npy

<hr style="border:2px solid black"></hr>


In [ ]:
# to generate new coefficients, run this cell
for j2 in tqdm(range (ngm)):
    phi, th2, th1 = angmat[j2,0],angmat[j2,1],angmat[j2,2]
    for j3 in range (lm):
        L1,L2,L = Lmat[j3,0],Lmat[j3,1],Lmat[j3,2]
        pxc = Bispher_SF(L1,L2,L, phi, th2, th1) 
        px[j2,j3]=pxc   

np.save(out_data + "4D_BiSp_coeff.npy", px)    # save Bispherical Harmonics coefficients to numpy readable file for future use

In [ ]:
# to load previously saved Legendre coefficients, run this cell
px = np.load("4D_BiSp_coeff.npy")    # save Bispherical Harmonics coefficients to numpy readable file for future use

<hr style="border:2px solid black"></hr>

### Running loop over full range of angular terms to calculate bispherical harmonics coefficients
* The pseudo-inverse is multiplied by E values at each R point (for all angular coordinates sorted similarly)
***
<span style="color:indigo"><u> Example output</u></span>

    Prints contents of df_Vnf (DataFrame containing radial coefficients) around minima region.
    Prints maxima and minima for isotropic and first anisotropic term. 
    The final radial coefficients with R values (first row) are saved in data/2D_MP/4D_Vlam_6_4.dat
***
To get lambda terms for each radial coefficient refer to data/4D_MP/Lambda_ref.dat 
<hr style="border:2px solid black"></hr>


In [ ]:
C_inv = np.linalg.pinv(px)
for i in range (Rpt):           # loop over all R 
    ct = i*ngm                  # counter
    f = df_inp.E[ct:ct+ngm]
    V_n1 = C_inv@(f)
    V_nf[i,:] = V_n1

a12 = np.arange(lm)             # a12 has same ordering as Lmat1
df_Vnf = pd.DataFrame(V_nf, columns = a12)
df_Vnf.insert(0, 'R', R_arr)                              # adding R column

print("The 15th to 30th terms are: \n ", df_Vnf[15:30])    # prints first few terms

df_Vnf.to_csv(out_data + '4D_Vlam_6_4.dat', index=None, header=True,
                sep=',')                              # save V_lam coefficients to file separated by comma

print("minima & maxima for isotropic term: ", 
      min(df_Vnf[0]), max(df_Vnf[0]))                 # prints minima & maxima for isotropic term
print("minima & maxima for first anisotropic term: ", 
      min(df_Vnf[1*sym]), max(df_Vnf[1*sym]))         # prints minima & maxima for first anisotropic term


<hr style="border:2px solid black"></hr>

### Read radial coefficients from a previously saved file (Optional/Commented)
The next lines can be used to load and plot previously saved radial coefficients in data/4D_MP/
<hr style="border:2px solid black"></hr>

In [ ]:
# # Read V_lam coefficients from file
# df_Vnf = pd.read_csv(out_data + '4D_Vlam_6_4.dat',sep=',',header=None)
# R_arr = df.iloc[:, 0].values    # merging two rows to create final R coordinates
# df_Vnf.drop(0,inplace= True)
# df_Vnf[15:30] # view minima region 

<hr style="border:2px solid black"></hr>

### Below is the basic (template) code to plot and fit the radial terms into MOLSCAT readable functions. 
#### Alternatively, one can load "4D_Vlam_6_4.dat" into CODE 4 and use more functions to fit the code.
* The function given below is a sum of three Slater functions of increasing order where coefficients a, b and c are optimised based on the data.
* Use as template - A good fit will require tweaking in order and number of functions. 
    * Too many functions will overfit the data may behave poorly with high energy region 
    * Less functions will underfit the data and give poor description of minima.

*** 
<span style="color:green"><u> Example Input</u></span>
     
    strt: x                 # x is the starting point of fit: including +/- infinities results in a poor fit.
***
<span style="color:indigo"><u> Example Output</u></span>
   
       Plots each radial term individually (at different x, y limits) and saves in /plots/4D_MP/
       Plots all V_lambda combined and saves in /plots/4D_MP/
       X axis is in Angstroms and Y is in cm-1. 
       
<hr style="border:2px solid black"></hr>


In [ ]:
from scipy.optimize import curve_fit

a,b,c,d,e,f,rmsx = np.zeros(lm),np.zeros(lm),np.zeros(lm),np.zeros(lm),np.zeros(lm),np.zeros(lm),np.zeros(lm)
def exp_fit(x, a,b,c):
    return  a*np.exp(-1*x)+b*np.exp(-2*x)+c*np.exp(-3*x)

In [ ]:
strt=15     # starting point for fit (choose carefully)
for i in range(0,lm):               # loop over V_lambda terms
    y_dummy = df_Vnf[i*sym]         # stores individual V_lambda for each loop
    plt.figure(figsize=(11,3))      # size of figure

    # curve fitting 
    parsx, covx = curve_fit(f=exp_fit, xdata=x_dummy[strt:], ydata=y_dummy[strt:], p0=[0,0,1000])
    a[j],b[j],c[j] = parsx
    print("No. :", i, " -- Radial coefficient [l1 l2 l]: ", Lmat[i])
    print("Fitting coefficients for exp_fit function [a b c]: ",parsx)

    # Plot each V_lambda_separately to view features and fitting 
    plt.subplot(1,3,1)                      # first subplot at visually appropriate x, y limit
    plt.scatter(R_arr, y_dummy,s=20, color='#00b3b3',
                label = 'no fit')               # scatter plot (blue) with label     
    plt.plot(xdummy, exp_fit(xdummy, *parsx), linestyle='-.', linewidth=2, 
             color='black', label = 'exp fit')  # fitted plot

    plt.grid(True,linestyle=':')                # grid on
    plt.minorticks_on()                         # minor ticks are on
    plt.title("V_lambda = %d" %(Lmat[i]))       # title of plot
    plt.ylim(max(y_dummy.min()-10,-200), 100)   # y limit
    plt.xlim(3, R_arr.max())                    # x limit
    
    plt.subplot(1,3,2)                      # second subplot at maximum x, y limit
    plt.scatter(R_arr, y_dummy,s=20, color='#00b3b3',label = 'no fit')
    plt.plot(xdummy, exp_fit(xdummy, *parsx), linestyle='-.', linewidth=2, color='black', label = 'exp fit')
    plt.grid(True,linestyle=':')
    plt.minorticks_on() 
    plt.title("V_lambda = %d" %(Lmat[i]))
    plt.ylim(y_dummy.min(), y_dummy.max())
    plt.xlim(R_arr.min()-0.5, R_arr.max())

    plt.subplot(1,3,3)                      # third subplot at zoomed in y limit [-1, +1]
    plt.scatter(R_arr, y_dummy,s=20, color='#00b3b3',label = 'no fit')
    plt.plot(xdummy, exp_fit(xdummy, *parsx), linestyle='-.', linewidth=2, color='black', label = 'exp fit')
    plt.grid(True,linestyle=':')
    plt.minorticks_on() 
    plt.title("V_lambda = %d" %(Lmat[i]))
    plt.ylim(-1, 1)
    plt.xlim(2.5, R_arr.max())
    plt.savefig(out_plots+'V_lam_{}.eps'.format(i*sym), 
                format='eps')               # save individual figure
    
    plt.tight_layout()                      # tight layout
    plt.show()                              # show individual plot
    print('-----'*20)
    print('\n')
    print('Double exponential RMSE = ',np.sqrt(np.average(np.power((exp_fit(x_dummy[strt:], 
                                                                            *parsx) - y_dummy[strt:]),2))))
    rmsx[j]=np.sqrt(np.average(np.power((exp_fit(x_dummy[strt:], *parsx) - y_dummy[strt:]),2)))
print('Average fit RMSE = ',np.average(rmsx))

# for j in range(0,lm):
#     y_dummy = df_Vnf[j]
#     parsx, covx = curve_fit(f=exp_fit, xdata=x_dummy[strt:], ydata=y_dummy[strt:], p0=[0,0,1000])
#     a[j],b[j],c[j] = parsx
#     print("Radial coefficient [l1 l2 l]: ", Lmat[i])
#     print("Fitting coefficients for exp_fit function [a b c]: ",parsx)
    
#     # Plot the fit data as an overlay on the scatter data
#     plt.scatter(x_dummy, y_dummy,s=20, color='#00b3b3',label = 'no fit')
    
#     # new range for fitted curve
#     xdummy=np.arange(2.5,15.1,0.1)
#     plt.plot(xdummy, exp_fit(xdummy, *parsx), linestyle='-.', linewidth=2, color='black', label = 'exp fit')
#     plt.legend(loc="upper right")
#     plt.ylabel("Energy (cm^(-1))")
#     plt.xlabel("R (Ang)")
#     plt.axhline(y=0, color='grey', linestyle=':')
#     plt.title("V_lambda = %d" %(i))
#     plt.ylim(max(y_dummy.min()-10,-200), 100)
#     plt.xlim(2.5, 15)
#     plt.show()
#     print('Double exponential RMSE = ',np.sqrt(np.average(np.power((exp_fit(x_dummy[strt:], 
#                                                                             *parsx) - y_dummy[strt:]),2))))
#     print('-----'*20)
#     print('\n')
#     rmsx[j]=np.sqrt(np.average(np.power((exp_fit(x_dummy[strt:], *parsx) - y_dummy[strt:]),2)))
# print('Average fit RMSE = ',np.average(rmsx))

<hr style="border:2px solid black"></hr>

### Save data for further use 
    1) either directly into MOLSCAT using VSTAR/VRTP (USE: 4D_Vlam_6_4.dat)
    2) OR curve fit into appropriate functions and then use in MOLSCAT (&POTL) 

#### If the above function results in poor fit, see CODE 4.
<hr style="border:2px solid black"></hr>

In [ ]:
# save output for each V lambdas as required by molscat!
print('LAMBDA ='),
for j in range (lm):
    print(Lmat[j,0],',',Lmat[j,1],',',Lmat[j,2],',')
print('NTERM  = ', '3,'*lm)
print('NPOWER = ', '0,0,0'*int(lm))
print('A      = ')
for j in range (lm):
    print(a[j],',',b[j],',',c[j],',')
print('E      =', '-1,-2,-3,'*lm) # change as per funtion taken above

<hr style="border:2px solid black"></hr>

## For more functions/options to fit radial coefficients, see: 
### CODE_4_MP_fit.ipynb
    The jupyter-notebook contains 3 common functions and fits 2D radial coefficients.
<hr style="border:2px solid black"></hr>
